The goal of this notebook is to see if we can witness a non-monotonous shape of the metric dimension in real graphs.

In [ ]:
import networkx as nx
import sys
from tqdm import tqdm
import plotly.graph_objs as go
sys.path.append('../ICH-algo')
from multilateration import *

### Copenhagen BT graph

In [ ]:
# Import graph such that we only keep the first edge between 2 vertices
G_bt = nx.Graph()

# read the file and add edges to the graph
with open('Copenhagen graphs/bt.csv/edges.csv', 'r') as f:
    next(f) # skip the first line

    for line in f:
        source, target, timestamp, _ = map(int, line.strip().split(','))

        # check if the edge already exists
        if G_bt.has_edge(source, target):
            existing_timestamp = G_bt[source][target]['timestamp']
            if timestamp < existing_timestamp:
                # remove the old edge and add the new one
                G_bt.remove_edge(source, target)
                G_bt.add_edge(source, target, timestamp=timestamp)
        else:
            # add the edge
            G_bt.add_edge(source, target, timestamp=timestamp)
            
# print the number of nodes and edges
print('Number of nodes:', G_bt.number_of_nodes())
print('Number of edges:', G_bt.number_of_edges())

We want to have a connected graph, hence, we will keep only the vertices that are added to the largest component before a given time. In the following cells, we explore what is the optimal largest component.

In [ ]:
# Sort edges by time
sorted_edges_bt = sorted(G_bt.edges(data=True), key=lambda x: x[2]['timestamp'])

# add property to the edges
for i, edge in enumerate(sorted_edges_bt):
    G_bt.edges[edge[:2]]['num_smaller_times'] = i

sorted_edges_bt = sorted(G_bt.edges(data=True), key=lambda x: x[2]['num_smaller_times'])

In [ ]:
# Check where it could be good to stop

nb_edges = [nb for nb in range(0, len(G_bt.edges), 100)]
max_component_size = []

for nb in tqdm(nb_edges):
    
    # create a new graph with the filtered edges
    G_f = nx.Graph()
    G_f.add_nodes_from(G_bt.nodes)
    G_f.add_edges_from(sorted_edges_bt[:nb])
    components = list(nx.connected_components(G_f))

    # find the largest component
    largest_component = max(components, key=len)
    max_component_size.append(len(largest_component))
    
    
# Define the trace for the scatter plot
trace = go.Scatter(x=nb_edges, y=max_component_size, mode='markers+lines')

# Define the layout
layout = go.Layout(title='Size of the largest component as a function of the number of edges', 
                   title_x=0.5,
                   xaxis=dict(title='Number of edges'), 
                   yaxis=dict(title='Largest connected component size'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

Using the elbow method, we set the number of edges and remove the nodes added to the largest component after this time.

In [ ]:
nb = 5800

In [ ]:
G_f = nx.Graph()
G_f.add_nodes_from(G_bt.nodes)
G_f.add_edges_from(sorted_edges_bt[:nb])
components = list(nx.connected_components(G_f))
list_to_remove = [el for i in components[1:] for el in i]
G_f.remove_nodes_from(list_to_remove)
G_bt.remove_nodes_from(list_to_remove)

In [ ]:
# sort edges by time
sorted_edges_bt = sorted(G_bt.edges(data=True), key=lambda x: x[2]['timestamp'])

# add property to the edges
for i, edge in enumerate(sorted_edges_bt):
    G_bt.edges[edge[:2]]['num_smaller_times'] = i

sorted_edges_bt = sorted(G_bt.edges(data=True), key=lambda x: x[2]['num_smaller_times'])

In [ ]:
result_nb_e_bt = {}
for t in tqdm(range(len(G_f.edges), len(G_bt.edges), 2000)):

    # create a new graph with the filtered edges
    G_f = nx.Graph()
    G_f.add_nodes_from(G_bt.nodes)
    G_f.add_edges_from(sorted_edges_bt[:t])
    a = len(ich(G_f))
    result_nb_e_bt[t] = a

In [ ]:
# Define your data
x = list(result_nb_e_bt.keys())
y = list(result_nb_e_bt.values())

# Define the trace for the scatter plot
trace = go.Scatter(x=x, y=y, mode='markers+lines')

# Define the layout
layout = go.Layout(#title='Probability of resolving the graph as a function of the subset cardinality', 
                   #title_x=0.5,
                   xaxis=dict(title='Number of edges at this given time'), 
                   yaxis=dict(title='Estimation of Metric Dimension'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

### Copenhagen sms graph

In [ ]:
# Import graph such that we only keep the first edge between 2 vertices
G_sms = nx.Graph()

# read the file and add edges to the graph
with open('Copenhagen graphs/sms.csv/edges.csv', 'r') as f:
    next(f) # skip the first line

    for line in f:
        source, target, timestamp = map(int, line.strip().split(','))

        # check if the edge already exists
        if G_sms.has_edge(source, target):
            existing_timestamp = G_sms[source][target]['timestamp']
            if timestamp < existing_timestamp:
                # remove the old edge and add the new one
                G_sms.remove_edge(source, target)
                G_sms.add_edge(source, target, timestamp=timestamp)
        else:
            # add the edge
            G_sms.add_edge(source, target, timestamp=timestamp)
# print the number of nodes and edges
print('Number of nodes:', G_sms.number_of_nodes())
print('Number of edges:', G_sms.number_of_edges())

We want to have a connected graph, hence, we will keep only the vertices that are added to the largest component before a given time. In the following cells, we explore what is the optimal largest component.

In [ ]:
# sort edges by time
sorted_edges_sms = sorted(G_sms.edges(data=True), key=lambda x: x[2]['timestamp'])

# add property to the edges
for i, edge in enumerate(sorted_edges_sms):
    G_sms.edges[edge[:2]]['num_smaller_times'] = i

sorted_edges_sms = sorted(G_sms.edges(data=True), key=lambda x: x[2]['num_smaller_times'])

In [ ]:
# Check where it could be good to stop

nb_edges = [nb for nb in range(0, len(G_sms.edges), 1)]
max_component_size = []

for nb in tqdm(nb_edges):
    
    # create a new graph with the filtered edges
    G_sub = nx.Graph()
    G_sub.add_nodes_from(G_sms.nodes)
    G_sub.add_edges_from(sorted_edges_sms[:nb])
    components = list(nx.connected_components(G_sub))

    # find the largest component
    largest_component = max(components, key=len)
    max_component_size.append(len(largest_component))

In [ ]:
# Define the trace for the scatter plot
trace = go.Scatter(x=nb_edges, y=max_component_size, mode='markers+lines')

# Define the layout
layout = go.Layout(title='Size of the largest component as a function of the number of edges', 
                   title_x=0.5,
                   xaxis=dict(title='Number of edges'), 
                   yaxis=dict(title='Largest connected component size'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

Using the elbow method, we set the number of edges and remove the nodes added to the largest component after this time.

In [ ]:
nb = 400 # hard to choose a good number here

In [ ]:
G_sub = nx.Graph()
G_sub.add_nodes_from(G_sms.nodes)
G_sub.add_edges_from(sorted_edges_sms[:nb])
components = list(nx.connected_components(G_sub))
list_to_remove = [el for i in components[1:] for el in i]
G_sub.remove_nodes_from(list_to_remove)
G_sms.remove_nodes_from(list_to_remove)

In [ ]:
# sort edges by time
sorted_edges_sms = sorted(G_sms.edges(data=True), key=lambda x: x[2]['timestamp'])

# add property to the edges
for i, edge in enumerate(sorted_edges_sms):
    G_sms.edges[edge[:2]]['num_smaller_times'] = i

sorted_edges_sms = sorted(G_sms.edges(data=True), key=lambda x: x[2]['num_smaller_times'])

In [ ]:
result_nb_e_sms = {}
for t in tqdm(range(len(G_sub.edges()), len(G_sms.edges()), 10)): #len(G_bt.edges)

    # create a new graph with the filtered edges
    G_f = nx.Graph()
    G_f.add_nodes_from(G_sms.nodes)
    G_f.add_edges_from(sorted_edges_sms[:t])
    a = len(ich(G_f))
    result_nb_e_sms[t] = a

In [ ]:
# Define your data
x = list(result_nb_e_sms.keys())
y = list(result_nb_e_sms.values())

# Define the trace for the scatter plot
trace = go.Scatter(x=x, y=y, mode='markers+lines')

# Define the layout
layout = go.Layout(#title='Probability of resolving the graph as a function of the subset cardinality', 
                   #title_x=0.5,
                   xaxis=dict(title='Number of edges at this given time'), 
                   yaxis=dict(title='Estimation of Metric Dimension'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

The obtained results are not highly persuasive as there is a presence of a non-monotonous zig-zag phenomenon in the metric dimension of real graphs.